In [1]:
import tensorflow as tf
import os
import numpy as np
import cv2
import csv
from tqdm import tqdm

In [8]:
def parse_data(filename, label) :
    img_read = tf.io.read_file(filename)
    img_decode = tf.image.decode_jpeg(img_read, channels=3)
#     img = tf.image.resize(img_decode, [227,227])
    return img_decode, label

def create_dataset(filenames, labels, batch_size) :
    dataset = tf.data.Dataset.from_tensor_slices((filenames,labels))
    dataset = dataset.map(parse_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=len(filenames))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    return dataset

def read_labels(path) :
    img_classes = os.listdir(path)
    total_len = len(os.listdir(path+img_classes[0])) + len(os.listdir(path+img_classes[1])) + \
    len(os.listdir(path+img_classes[2]))+ len(os.listdir(path+img_classes[3])) + \
    len(os.listdir(path+img_classes[4]))+ len(os.listdir(path+img_classes[5]))+\
    len(os.listdir(path+img_classes[6]))

    names = []
    labels = np.zeros((total_len, len(img_classes)))
    j=0
    for i, img_class in enumerate(img_classes) :
        for img in os.listdir(os.path.join(path, img_class)) :
            names.append(os.path.join(path, img_class, img))
            labels[j][i] = 1
            j+=1
        
    return names, labels


batch_size = 16
images, labels = read_labels('./train/')
dataset = create_dataset(images, labels, batch_size)

In [9]:
effib1 =  tf.keras.applications.EfficientNetB1(include_top=False, weights='imagenet', input_shape=(227,227,3))
effib1.summary()

Model: "efficientnetb1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 227, 227, 3) 0                                            
__________________________________________________________________________________________________
rescaling_2 (Rescaling)         (None, 227, 227, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
normalization_2 (Normalization) (None, 227, 227, 3)  7           rescaling_2[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 229, 229, 3)  0           normalization_2[0][0]            
_____________________________________________________________________________________

In [10]:
from keras import Sequential, layers, Model
model = Sequential()
model.add(effib1)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(7, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb1 (Functional)  (None, 8, 8, 1280)        6575239   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 8967      
Total params: 6,584,206
Trainable params: 6,522,151
Non-trainable params: 62,055
_________________________________________________________________


In [11]:
metrics = [
    tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='AUC')
]
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
             loss=tf.keras.losses.CategoricalCrossentropy(),
             metrics=metrics)

In [12]:
history = model.fit(dataset,
                    batch_size=16,
                    epochs=4)

Epoch 1/4
107/107 [==============================] - 596s 6s/step - loss: 0.6382 - accuracy: 0.7927 - precision: 0.8682 - recall: 0.7097 - AUC: 0.9645
Epoch 2/4
107/107 [==============================] - 587s 5s/step - loss: 0.2839 - accuracy: 0.8981 - precision: 0.9202 - recall: 0.8757 - AUC: 0.9914
Epoch 3/4
107/107 [==============================] - 587s 5s/step - loss: 0.2500 - accuracy: 0.9193 - precision: 0.9342 - recall: 0.9028 - AUC: 0.9923
Epoch 4/4
107/107 [==============================] - 609s 6s/step - loss: 0.2015 - accuracy: 0.9382 - precision: 0.9504 - recall: 0.9252 - AUC: 0.9954


In [13]:
from tqdm import tqdm
model.save_weights('programmers_v5.ht')
test_path = './test/0'

with open('test_answer5.csv', 'w' , newline='') as f:
    wr = csv.writer(f)
    wr.writerow(['','answer value'])
    for i, image in tqdm(enumerate(os.listdir(test_path))) :
        img = cv2.imread(os.path.join(test_path, image))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, dsize=(227,227))
        img = tf.reshape(img, (-1, img.shape[0], img.shape[1], img.shape[2]))

        pred = model.predict(img)
        answer = np.argmax(pred[0])
        wr.writerow([i, answer])
    

350it [00:32, 10.86it/s]
